In [6]:
import lancedb
db_path = "C:\\Lee\\work\\contract\\db\\tmp\\contact_vectors.lancedb"
db_path=r"C:/Lee/work/db/contact_lancedb"
# --- 1. 连接/创建 LanceDB 数据库 ---
db = lancedb.connect(db_path)
print(f"LanceDB connected at: {db.uri}")

LanceDB connected at: C:\Lee\work\db\contact_lancedb


In [7]:
# --- 2. 打开表 ---
table_name = "contact_table" # 你之前创建的表名
table_name = 'contract_table'
try:
    table = db.open_table(table_name)
    print(f"Successfully opened table: '{table.name}'")
    print(f"Table schema: {table.schema}")
    print(f"Number of rows in table: {len(table)}")
except Exception as e:
    print(f"Error opening table '{table_name}': {e}")
    print(f"Available tables: {db.table_names()}")
    exit()

Successfully opened table: 'contract_table'
Table schema: vector: list<item: float>
  child 0, item: float
meta_str: string
doc: string
contact_no: string
project_name: string
subitem_name: string
date: string
total_price_str: string
main_equipments: string
sub_equipments: string
supplier: string
year: int16
total_price: float
Number of rows in table: 32


In [4]:
import json

def read_and_decode_table(db, table_name):
    table = db.open_table(table_name)
    df = table.to_pandas()
    
    if 'payload' not in df.columns:
        raise ValueError("DataFrame中缺少'payload'列")
    
    # 解析payload字符串为字典
    df['parsed_payload'] = df['payload'].apply(
        lambda x: json.loads(x) if isinstance(x, str) else x
    )
    
    # 直接提取已解码的字段值
    df['name_cn'] = df['parsed_payload'].apply(
        lambda x: x.get('name') if isinstance(x, dict) else None
    )
    
    df['content_cn'] = df['parsed_payload'].apply(
        lambda x: x.get('content') if isinstance(x, dict) else None
    )
    
    # 可选：删除临时解析列
    df = df.drop(columns=['parsed_payload'])
    
    return df

In [5]:
data = read_and_decode_table(db, table_name)

ValueError: DataFrame中缺少'payload'列

In [5]:
data.iloc[0]['content_cn']

'买方合同编号：TGYR\\-2023\\-01\n\n卖方合同编号：\n\n订 货 合 同\n\n项目名称：山东泰山钢铁集团有限公司\n\n265㎡烧环冷机余热系统改造工程EPC总承包余热回收项目\n\n子项名称：环冷机余热锅炉\n\n买方：湖南中冶长天节能环保技术有限公司\n\n卖方：江苏东九重工股份有限公司\n\n合同签订时期：2023年 8 月\n\n合同签订地点：湖南·长沙\n\n合 同 双 方\n\n__买\u3000\u3000方（印章）__\n\n__卖\u3000\u3000方（印章）__\n\n__名   称：湖南中冶长天节能环保技术有限公司__\n\n__名  称：江苏东九重工股份有限公司__\n\n__地    址：长沙市岳麓区节庆路7号__\n\n__地址：盐城高新区华锐南路9号__\n\n__联 系 人：戴亮__\n\n__联 系 人：刘忠__\n\n__电    话：18773083301__\n\n__电    话：0515\\-88597534__\n\n__传    真：__\n\n__传    真：0515\\-88597987__\n\n__电子邮件：1203852335@qq\\.com__\n\n__电子邮件：Liuz118@163\\.com__\n\n__开户银行：浦发银行长沙东塘支行__\n\n__开户银行： 盐城市中行亭湖支行__\n\n__帐    号：6605 0155 2000 0246 8__\n\n__帐    号：5456 5820 7770__\n\n__纳税人登记号：9143 0100 5849 3952 00__\n\n__纳税人登记号：91320900140334109F__\n\n__单位名称：湖南中冶长天节能环保技术有限公司__\n\n__单位名称： 江苏东九重工股份有限公司__\n\n__委托代理人：__\n\n__委托代理人： __\n\n__签字日期：__\n\n__签字日期：__\n\n3\\.1  最终供货一览表\n\n__序号__\n\n__货物名称__\n\n__规格型号__\n\n__单位__\n\n__数量__\n\n__重量__\n\n__单   价__\n\n__（万元RMB）__\n\n__总   价__\n\n__（万元RMB）__\n\n__交 货 期__\n\n

In [14]:
len(data.iloc[0]['content_cn'])

1499

In [6]:
data

,vector,id,payload,name_cn,content_cn
0,"[0.021162663, -0.057701577, 0.04414577, -0.022...",c2f491f7f95725d3bb0fdf48f5599ccd,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,买方合同编号：TGYR\-2023\-01\n\n卖方合同编号：\n\n订 货 合 同\n\...
1,"[0.03296253, -0.040740207, 0.050524026, 0.0732...",36154422eadf8a461041543de6bae879,"{""name"": null, ""meta_data"": {}, ""content"": ""__...",None,__买方合同编号：SZWF\-CG\-2011\-001__\n\n__卖方合同编号：__\...
2,"[0.024683388, -0.06892115, 0.06433709, 0.02245...",883aa66877272f477802cd8a536b7d82,"{""name"": null, ""meta_data"": {}, ""content"": ""__...",None,__买方合同编号：BGYR3\-E\-2014\-01__\n\n__卖方合同编号：__\n...
3,"[0.029732762, -0.055134043, 0.059734143, -0.00...",f1387a7ec67c647a7aa144d84f40413e,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,买方合同编号：WGZJGLD\-2019\-01\n\n卖方合同编号：\n\n订 货 合 同...
4,"[0.02021305, -0.08162838, 0.04443637, -0.00688...",e4a48ccace620675bc60cbf939c8e206,"{""name"": null, ""meta_data"": {}, ""content"": ""__...",None,__买方合同编号：__BGYR3\-E\-2024\-01\n\n__卖方合同编号：__\n...
...,...,...,...,...,...
76,"[0.010763203, -0.066128105, 0.030851355, -0.00...",796c79715ce897de0924aa8cb0901d9e,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,买方合同编号：AYGTYQJH\-2017\-08\n\n卖方合同编号：\n\n订 货 合 ...
77,"[0.00504349, -0.06691239, 0.017229313, -0.0079...",b39258d8375294d53e9bf87217896735,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,买方合同编号：AYGTYQJH\-2017\-08补\n\n卖方合同编号：\n\n订 货 合...
78,"[-0.007343541, -0.09411233, 0.0275852, -0.0148...",253358c8987157d8098c8519dac37f06,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,买方合同编号：AYGTYQJH\-2017\-10\n\n卖方合同编号：\n\n订 货 合 ...
79,"[0.025154898, -0.066300765, 0.029478136, 0.007...",a629a69e27f44c85369021438bcbf12e,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,买方合同编号：AYGTYQJH\-2017\-11\n\n卖方合同编号：\n\n订 货 合 ...
